### Comparing different clustering algorithms


* This notebook compares different clustering algorithms computation time and clustering performance. 
* 514 features were selected out of the 1063 features
* Three data processing approaches: 
* &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Approach I. Use PCA to reduce the original 1063 dimensions. Drop all the categorical values and impute the numerical values with means, the method will be applied on a ramdonly generated subsamples (1.25%) of the original size data.
* &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Approach II. All variables have been transformed into numerical values using low rank representation method, the reduced dimension matrix is used here to determine optimal K.
* &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Approach III. 514 features were selected out of the 1063 features. Then use LASSO and PCA to contidue reduce dimension to 44 variables (all of them turned out to be numerical variables). Impute NA with means.

Reference: 
http://scikit-learn.org/stable/modules/clustering.html

http://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html#sphx-glr-auto-examples-cluster-plot-cluster-comparison-py

In [1]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist, pdist
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.neighbors import kneighbors_graph
import matplotlib.cm as cm
from sklearn import cluster, mixture
from collections import defaultdict
from sklearn.decomposition import PCA
from sklearn_extensions.fuzzy_kmeans import KMedians, FuzzyKMeans, KMeans
import time, warnings
%matplotlib inline

#### Approach I. Use PCA to reduce the original 1063 dimensions. Drop all the categorical values and impute the numerical values with means, the method will be applied on a ramdonly generated subsamples (1.25%) of the original size data.

In [20]:
df = pd.read_csv('PHBsample14_sss.csv', low_memory=False)

In [21]:
# drop the column resulted from sampling of the original data set
df.drop('Unnamed: 0', axis=1, inplace=True)
# In order to run K-means, drop all the categoricald data for now.
df = df.select_dtypes(include=['float64', 'int64'])
# Impute missing values with means
df = df.fillna(df.mean())
pca = PCA(2, svd_solver='randomized')
pca.fit(df)
df_reduced = pca.fit_transform(df)
X = StandardScaler().fit_transform(df_reduced)

In [22]:
params = {'quantile': .3,
                'eps': .3,
                'damping': .9,
                'preference': -200,
                'n_neighbors': 10,
                'n_clusters': 7}

In [23]:
# estimate bandwidth for mean shift (unable to calculate on low rank data due to ram size limit, \
# therefore bandwidth was estimated on a sample data)
bandwidth = cluster.estimate_bandwidth(X, quantile=params['quantile'])

In [24]:
# connectivity matrix for structured Ward (unable to calculate on low rank data due to ram size limit)
connectivity = kneighbors_graph(
    X, n_neighbors=params['n_neighbors'], include_self=False)
# make connectivity symmetric
connectivity = 0.5 * (connectivity + connectivity.T)

In [25]:
kmeans = cluster.KMeans(n_clusters=params['n_clusters'], random_state=0)
kmedians = KMedians(k=params['n_clusters'])
fuzzy_kmeans = FuzzyKMeans(k=params['n_clusters'], m=2)
ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
two_means = cluster.MiniBatchKMeans(n_clusters=params['n_clusters'])
ward = cluster.AgglomerativeClustering(
    n_clusters=params['n_clusters'], linkage='ward',
    connectivity=connectivity)
spectral = cluster.SpectralClustering(
    n_clusters=params['n_clusters'], eigen_solver='arpack',
    affinity="nearest_neighbors")
dbscan = cluster.DBSCAN(eps=params['eps'])
affinity_propagation = cluster.AffinityPropagation(
    damping=params['damping'], preference=params['preference'])
average_linkage = cluster.AgglomerativeClustering(
    linkage="average", affinity="cityblock",
    n_clusters=params['n_clusters'], connectivity=connectivity)
birch = cluster.Birch(n_clusters=params['n_clusters'])
gmm = mixture.GaussianMixture(
    n_components=params['n_clusters'], covariance_type='full')

clustering_algorithms = (
        ('KMeans', kmeans),
        ('kMedians', kmedians),
        ('fuzzy KMeans', fuzzy_kmeans),
        ('MiniBatchKMeans', two_means),
        ('MeanShift', ms),
        ('Ward', ward),
        ('AgglomerativeClustering', average_linkage),
        ('DBSCAN', dbscan),
        ('Birch', birch),
        ('GaussianMixture', gmm)
    )

In [26]:
res = defaultdict(list)
for name, algorithm in clustering_algorithms:
    t0 = time.time()
    # catch warnings related to kneighbors_graph
    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore",
            message="the number of connected components of the " +
            "connectivity matrix is [0-9]{1,2}" +
            " > 1. Completing it to avoid stopping the tree early.",
            category=UserWarning)
        warnings.filterwarnings(
            "ignore",
            message="Graph is not fully connected, spectral embedding" +
            " may not work as expected.",
            category=UserWarning)
    algorithm.fit(X)
    t1 = time.time()
    
    res[name].append(t1-t0)
    if name == 'KMeans' or name == 'MiniBatchKMeans':
        inertia = algorithm.inertia_
    else:
        inertia = 'N/A'
    res[name].append(inertia)
    if name == 'GaussianMixture':
        labels = algorithm.predict(X)
    else:
        labels = algorithm.labels_
        score = silhouette_score(X, labels, metric='euclidean', sample_size=3000)
    res[name].append(score)
    if name == 'MeanShift':
        labels_unique = np.unique(labels)
        K = len(labels_unique)     
    elif name == 'DBSCAN':
        K = len(set(labels)) - (1 if -1 in labels else 0)
    else:
        K = params['n_clusters']
    res[name].append(K)
    print(name + " done")

KMeans done
kMedians done
fuzzy KMeans done
MiniBatchKMeans done
MeanShift done


/home/capsops/.conda/envs/capsenvs/lib/python3.6/site-packages/sklearn/cluster/hierarchical.py:193: UserWarning: the number of connected components of the connectivity matrix is 41 > 1. Completing it to avoid stopping the tree early.
  affinity='euclidean')


Ward done


/home/capsops/.conda/envs/capsenvs/lib/python3.6/site-packages/sklearn/cluster/hierarchical.py:426: UserWarning: the number of connected components of the connectivity matrix is 41 > 1. Completing it to avoid stopping the tree early.
  affinity=affinity)


AgglomerativeClustering done
DBSCAN done
Birch done
GaussianMixture done


In [27]:
df = pd.DataFrame.from_dict(res, orient='index')
df.columns = ['Run time', 'Within-cluster sum of squares(inertia)', \
                                                         'Silhouette score', 'Number of clusters']

In [28]:
df

,Run time,Within-cluster sum of squares(inertia),Silhouette score,Number of clusters
KMeans,0.773058,7309.46,0.626288,7
kMedians,0.073632,N/A,0.519590,7
fuzzy KMeans,0.273170,N/A,0.642657,7
MiniBatchKMeans,0.333234,8087.54,0.608805,7
MeanShift,0.507678,N/A,0.540671,3
Ward,20.460196,N/A,0.635124,7
AgglomerativeClustering,12.801505,N/A,0.497204,7
DBSCAN,6.936930,N/A,0.486356,7
Birch,0.841233,N/A,0.599285,7
GaussianMixture,0.355665,N/A,0.599285,7


In [29]:
del df['Within-cluster sum of squares(inertia)']
del df['Number of clusters']
df

,Run time,Silhouette score
KMeans,0.773058,0.626288
kMedians,0.073632,0.519590
fuzzy KMeans,0.273170,0.642657
MiniBatchKMeans,0.333234,0.608805
MeanShift,0.507678,0.540671
Ward,20.460196,0.635124
AgglomerativeClustering,12.801505,0.497204
DBSCAN,6.936930,0.486356
Birch,0.841233,0.599285
GaussianMixture,0.355665,0.599285


In [30]:
df.ix['DBSCAN', 'Silhouette score'] = 'N/A'
df.ix['GaussianMixture', 'Silhouette score'] = 'N/A'
df

/home/capsops/.conda/envs/capsenvs/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,Run time,Silhouette score
KMeans,0.773058,0.626288
kMedians,0.073632,0.51959
fuzzy KMeans,0.273170,0.642657
MiniBatchKMeans,0.333234,0.608805
MeanShift,0.507678,0.540671
Ward,20.460196,0.635124
AgglomerativeClustering,12.801505,0.497204
DBSCAN,6.936930,N/A
Birch,0.841233,0.599285
GaussianMixture,0.355665,N/A


#### Approach II. All variables have been transformed into numerical values using low rank representation method, the reduced dimension matrix is used here to determine optimal K.

In [23]:
df = pd.read_csv('/mnt/UW/outputDataset/lowrank_rep.csv.gz')

In [24]:
X = StandardScaler().fit_transform(df)

In [5]:
params = {'quantile': .3,
                'eps': .3,
                'damping': .9,
                'preference': -200,
                'n_neighbors': 10,
                'n_clusters': 7}

In [3]:
# Use a sample data set to estimate bandwidth and connectivity 
df_sample = pd.read_csv('PHBsample14_sss.csv', low_memory=False)
# drop the column resulted from sampling of the original data set
df_sample.drop('Unnamed: 0', axis=1, inplace=True)
# In order to run K-means, drop all the categoricald data for now.
df_sample = df_sample.select_dtypes(include=['float64', 'int64'])
# Impute missing values with means
df_sample = df_sample.fillna(df_sample.mean())

In [10]:
pca = PCA(2, svd_solver='randomized')
pca.fit(df_sample)
df_reduced = pca.fit_transform(df_sample)
X_sample = StandardScaler().fit_transform(df_reduced)

In [11]:
# estimate bandwidth for mean shift (unable to calculate on low rank data due to ram size limit, \
# therefore bandwidth was estimated on a sample data)
bandwidth = cluster.estimate_bandwidth(X_sample, quantile=params['quantile'])

In [31]:
from sklearn_extensions.fuzzy_kmeans import KMedians, FuzzyKMeans, KMeans
    
kmeans = cluster.KMeans(n_clusters=params['n_clusters'], random_state=0)
kmedians = KMedians(k=params['n_clusters'])
fuzzy_kmeans = FuzzyKMeans(k=params['n_clusters'], m=2)
ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
two_means = cluster.MiniBatchKMeans(n_clusters=params['n_clusters'])
ward = cluster.AgglomerativeClustering(
    n_clusters=params['n_clusters'], linkage='ward',
    connectivity=connectivity)
spectral = cluster.SpectralClustering(
    n_clusters=params['n_clusters'], eigen_solver='arpack',
    affinity="nearest_neighbors")
dbscan = cluster.DBSCAN(eps=params['eps'])
affinity_propagation = cluster.AffinityPropagation(
    damping=params['damping'], preference=params['preference'])
average_linkage = cluster.AgglomerativeClustering(
    linkage="average", affinity="cityblock",
    n_clusters=params['n_clusters'], connectivity=connectivity)
birch = cluster.Birch(n_clusters=params['n_clusters'])
gmm = mixture.GaussianMixture(
    n_components=params['n_clusters'], covariance_type='full')

clustering_algorithms = (
        ('KMeans', kmeans),
        ('kMedians', kmedians),
        ('fuzzy KMeans', fuzzy_kmeans),
        ('MiniBatchKMeans', two_means),
        #('MeanShift', ms),
        #('Ward', ward),
        #('AgglomerativeClustering', average_linkage),
        ('DBSCAN', dbscan),
        #('Birch', birch),
        ('GaussianMixture', gmm)
    )

In [32]:
res = defaultdict(list)
for name, algorithm in clustering_algorithms:
    t0 = time.time()
    # catch warnings related to kneighbors_graph
    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore",
            message="the number of connected components of the " +
            "connectivity matrix is [0-9]{1,2}" +
            " > 1. Completing it to avoid stopping the tree early.",
            category=UserWarning)
        warnings.filterwarnings(
            "ignore",
            message="Graph is not fully connected, spectral embedding" +
            " may not work as expected.",
            category=UserWarning)
    algorithm.fit(X)
    t1 = time.time()
    
    res[name].append(t1-t0)
    if name == 'KMeans' or name == 'MiniBatchKMeans':
        inertia = algorithm.inertia_
    else:
        inertia = 'N/A'
    res[name].append(inertia)
    if name == 'GaussianMixture':
        labels = algorithm.predict(X)
    else:
        labels = algorithm.labels_
        score = silhouette_score(X, labels, metric='euclidean', sample_size=3000)
    res[name].append(score)
    if name == 'MeanShift':
        labels_unique = np.unique(labels)
        K = len(labels_unique)     
    elif name == 'DBSCAN':
        K = len(set(labels)) - (1 if -1 in labels else 0)
    else:
        K = params['n_clusters']
    res[name].append(K)
    print(name + " done")

KMeans done
kMedians done
fuzzy KMeans done
MiniBatchKMeans done
DBSCAN done
GaussianMixture done


In [33]:
res

defaultdict(list,
            {'DBSCAN': [1577.0902400016785, 'N/A', -0.3141345830264103, 22330],
             'GaussianMixture': [464.31207513809204,
              'N/A',
              -0.3141345830264103,
              7],
             'KMeans': [369.5000250339508,
              198120505.7412454,
              0.06076279554643035,
              7],
             'MiniBatchKMeans': [40.39845132827759,
              197757397.62792787,
              0.06060075801589179,
              7],
             'fuzzy KMeans': [6.142011642456055,
              'N/A',
              0.005012715651733739,
              7],
             'kMedians': [103.91523027420044, 'N/A', 0.054670054511030956, 7]})

In [34]:
df = pd.DataFrame.from_dict(res, orient='index')
df.columns = ['Run time', 'Within-cluster sum of squares(inertia)', \
                                                         'Silhouette score', 'Number of clusters']

In [35]:
df

,Run time,Within-cluster sum of squares(inertia),Silhouette score,Number of clusters
KMeans,369.500025,1.98121e+08,0.060763,7
kMedians,103.915230,N/A,0.054670,7
fuzzy KMeans,6.142012,N/A,0.005013,7
MiniBatchKMeans,40.398451,1.97757e+08,0.060601,7
DBSCAN,1577.090240,N/A,-0.314135,22330
GaussianMixture,464.312075,N/A,-0.314135,7


In [36]:
del df['Within-cluster sum of squares(inertia)']
del df['Number of clusters']
df

,Run time,Silhouette score
KMeans,369.500025,0.060763
kMedians,103.915230,0.054670
fuzzy KMeans,6.142012,0.005013
MiniBatchKMeans,40.398451,0.060601
DBSCAN,1577.090240,-0.314135
GaussianMixture,464.312075,-0.314135


#### Approach III 514 features were selected out of the 1063 features. Then use LASSO and PCA to contidue reduce dimension to 44 variables (all of them turned out to be numerical variables). Impute NA with means.

In [2]:
df = pd.read_csv('/home/capsops/mandy/pca_reduced_LASSO.csv')

In [3]:
# records number is too large, use a sample to perform analysis
import random
random.seed(123)
df_sample = df.sample(frac=0.0125, replace=False)
df_sample = df_sample.fillna(df_sample.mean())
X = StandardScaler().fit_transform(df_sample)

In [4]:
params = {'quantile': .3,
                'eps': .3,
                'damping': .9,
                'preference': -200,
                'n_neighbors': 10,
                'n_clusters': 7}

In [5]:
# estimate bandwidth for mean shift (unable to calculate on low rank data due to ram size limit, \
# therefore bandwidth was estimated on a sample data)
bandwidth = cluster.estimate_bandwidth(X, quantile=params['quantile'])

In [8]:
# connectivity matrix for structured Ward (unable to calculate on low rank data due to ram size limit)
connectivity = kneighbors_graph(
    X, n_neighbors=params['n_neighbors'], include_self=False)
# make connectivity symmetric
connectivity = 0.5 * (connectivity + connectivity.T)

In [11]:
from sklearn_extensions.fuzzy_kmeans import KMedians, FuzzyKMeans, KMeans
    
kmeans = cluster.KMeans(n_clusters=params['n_clusters'], random_state=0)
kmedians = KMedians(k=params['n_clusters'])
fuzzy_kmeans = FuzzyKMeans(k=params['n_clusters'], m=2)
ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
two_means = cluster.MiniBatchKMeans(n_clusters=params['n_clusters'])
ward = cluster.AgglomerativeClustering(
    n_clusters=params['n_clusters'], linkage='ward',
    connectivity=connectivity)
spectral = cluster.SpectralClustering(
    n_clusters=params['n_clusters'], eigen_solver='arpack',
    affinity="nearest_neighbors")
dbscan = cluster.DBSCAN(eps=params['eps'])
affinity_propagation = cluster.AffinityPropagation(
    damping=params['damping'], preference=params['preference'])
average_linkage = cluster.AgglomerativeClustering(
    linkage="average", affinity="cityblock",
    n_clusters=params['n_clusters'], connectivity=connectivity)
birch = cluster.Birch(n_clusters=params['n_clusters'])
gmm = mixture.GaussianMixture(
    n_components=params['n_clusters'], covariance_type='full')

clustering_algorithms = (
        ('KMeans', kmeans),
        ('kMedians', kmedians),
        ('fuzzy KMeans', fuzzy_kmeans),
        ('MiniBatchKMeans', two_means),
        ('MeanShift', ms),
        ('Ward', ward),
        #('AgglomerativeClustering', average_linkage),
        ('DBSCAN', dbscan),
        ('Birch', birch),
        ('GaussianMixture', gmm)
    )

In [12]:
res = defaultdict(list)
for name, algorithm in clustering_algorithms:
    t0 = time.time()
    # catch warnings related to kneighbors_graph
    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore",
            message="the number of connected components of the " +
            "connectivity matrix is [0-9]{1,2}" +
            " > 1. Completing it to avoid stopping the tree early.",
            category=UserWarning)
        warnings.filterwarnings(
            "ignore",
            message="Graph is not fully connected, spectral embedding" +
            " may not work as expected.",
            category=UserWarning)
    algorithm.fit(X)
    t1 = time.time()
    
    res[name].append(t1-t0)
    if name == 'KMeans' or name == 'MiniBatchKMeans':
        inertia = algorithm.inertia_
    else:
        inertia = 'N/A'
    res[name].append(inertia)
    if name == 'GaussianMixture':
        labels = algorithm.predict(X)
    else:
        labels = algorithm.labels_
        score = silhouette_score(X, labels, metric='euclidean', sample_size=3000)
    res[name].append(score)
    if name == 'MeanShift':
        labels_unique = np.unique(labels)
        K = len(labels_unique)     
    elif name == 'DBSCAN':
        K = len(set(labels)) - (1 if -1 in labels else 0)
    else:
        K = params['n_clusters']
    res[name].append(K)
    print(name + " done")

KMeans done
kMedians done
fuzzy KMeans done
MiniBatchKMeans done
MeanShift done
Ward done
DBSCAN done
Birch done
GaussianMixture done


In [13]:
res

defaultdict(list,
            {'Birch': [214.02572512626648, 'N/A', 0.0491878871190508, 7],
             'DBSCAN': [112.28516745567322, 'N/A', -0.32693649695384797, 3],
             'GaussianMixture': [30.90679955482483,
              'N/A',
              0.0491878871190508,
              7],
             'KMeans': [6.830701112747192,
              2208482.103363145,
              0.02248782135579389,
              7],
             'MeanShift': [71.05786800384521, 'N/A', 0.3595852110844424, 14],
             'MiniBatchKMeans': [0.26132869720458984,
              2268486.0900941584,
              0.017860876584572003,
              7],
             'Ward': [33.082340478897095, 'N/A', 0.038806970243843705, 7],
             'fuzzy KMeans': [0.1179192066192627,
              'N/A',
              -0.02081601275773363,
              7],
             'kMedians': [2.5176849365234375, 'N/A', 0.01355521541189735, 7]})

In [14]:
df = pd.DataFrame.from_dict(res, orient='index')
df.columns = ['Run time', 'Within-cluster sum of squares(inertia)', \
                                                         'Silhouette score', 'Number of clusters']
df

,Run time,Within-cluster sum of squares(inertia),Silhouette score,Number of clusters
KMeans,6.830701,2.20848e+06,0.022488,7
kMedians,2.517685,N/A,0.013555,7
fuzzy KMeans,0.117919,N/A,-0.020816,7
MiniBatchKMeans,0.261329,2.26849e+06,0.017861,7
MeanShift,71.057868,N/A,0.359585,14
Ward,33.082340,N/A,0.038807,7
DBSCAN,112.285167,N/A,-0.326936,3
Birch,214.025725,N/A,0.049188,7
GaussianMixture,30.906800,N/A,0.049188,7


In [15]:
del df['Within-cluster sum of squares(inertia)']
del df['Number of clusters']
df

,Run time,Silhouette score
KMeans,6.830701,0.022488
kMedians,2.517685,0.013555
fuzzy KMeans,0.117919,-0.020816
MiniBatchKMeans,0.261329,0.017861
MeanShift,71.057868,0.359585
Ward,33.082340,0.038807
DBSCAN,112.285167,-0.326936
Birch,214.025725,0.049188
GaussianMixture,30.906800,0.049188


In [2]:
res = defaultdict(list,
            {'Birch': [214.02572512626648, 'N/A', 0.0491878871190508, 7],
             'DBSCAN': [112.28516745567322, 'N/A', -0.32693649695384797, 3],
             'GaussianMixture': [30.90679955482483,
              'N/A',
              0.0491878871190508,
              7],
             'KMeans': [6.830701112747192,
              2208482.103363145,
              0.02248782135579389,
              7],
             'MeanShift': [71.05786800384521, 'N/A', 0.3595852110844424, 14],
             'MiniBatchKMeans': [0.26132869720458984,
              2268486.0900941584,
              0.017860876584572003,
              7],
             'Ward': [33.082340478897095, 'N/A', 0.038806970243843705, 7],
             'fuzzy KMeans': [0.1179192066192627,
              'N/A',
              -0.02081601275773363,
              7],
             'kMedians': [2.5176849365234375, 'N/A', 0.01355521541189735, 7]})
df = pd.DataFrame.from_dict(res, orient='index')
df.columns = ['Run time', 'Within-cluster sum of squares(inertia)', \
                                                         'Silhouette score', 'Number of clusters']
df

,Run time,Within-cluster sum of squares(inertia),Silhouette score,Number of clusters
Birch,214.025725,N/A,0.049188,7
DBSCAN,112.285167,N/A,-0.326936,3
GaussianMixture,30.906800,N/A,0.049188,7
KMeans,6.830701,2.20848e+06,0.022488,7
MeanShift,71.057868,N/A,0.359585,14
MiniBatchKMeans,0.261329,2.26849e+06,0.017861,7
Ward,33.082340,N/A,0.038807,7
fuzzy KMeans,0.117919,N/A,-0.020816,7
kMedians,2.517685,N/A,0.013555,7


In [3]:
del df['Within-cluster sum of squares(inertia)']
del df['Number of clusters']
df

,Run time,Silhouette score
Birch,214.025725,0.049188
DBSCAN,112.285167,-0.326936
GaussianMixture,30.906800,0.049188
KMeans,6.830701,0.022488
MeanShift,71.057868,0.359585
MiniBatchKMeans,0.261329,0.017861
Ward,33.082340,0.038807
fuzzy KMeans,0.117919,-0.020816
kMedians,2.517685,0.013555
